In [0]:
%skip
%sql
-- Use your own catalog / schema if needed
CREATE SCHEMA IF NOT EXISTS demo_delta;
USE demo_delta;

-- Clean up from previous runs
DROP TABLE IF EXISTS sales_parquet;
DROP TABLE IF EXISTS sales_delta;
DROP TABLE IF EXISTS sales_delta_bad;


In [0]:
%sql
-- Create a simple sales dataset
CREATE OR REPLACE TEMP VIEW sales_source AS
SELECT
  1 AS id, "2026-01-01" AS sale_date, "UK" AS country, 100.0 AS amount UNION ALL
SELECT 2, "2026-01-01", "US", 200.0 UNION ALL
SELECT 3, "2026-01-02", "IN", 300.0;


In [0]:
%sql
DROP TABLE IF EXISTS sales_delta;
CREATE TABLE sales_delta
USING DELTA
AS
SELECT * FROM sales_source;

In [0]:
%sql

SELECT * FROM sales_delta;

In [0]:
%sql 
DESCRIBE TABLE sales_delta;

In [0]:
%sql
DESCRIBE HISTORY sales_delta;


In [0]:
%sql

-- Update example: increase UK sales by 10%
UPDATE sales_delta
SET amount = amount * 1.1
WHERE country = 'UK';

In [0]:
%sql

SELECT * FROM sales_delta;

In [0]:
%sql
DESCRIBE HISTORY sales_delta;

In [0]:
%sql
-- Delete example: simulate GDPR delete
DELETE FROM sales_delta
WHERE id = 2;


In [0]:
%sql

SELECT * FROM sales_delta;

In [0]:
%sql
DESCRIBE HISTORY sales_delta;

In [0]:
%sql

SELECT * FROM sales_delta;

In [0]:
%sql
-- Using VERSION AS OF
SELECT * FROM sales_delta VERSION AS OF 0;

-- Or using timestamp if you prefer
-- SELECT * FROM sales_delta TIMESTAMP AS OF '2025-01-01T12:00:00Z';


In [0]:
%sql
-- Simulate a streaming-like append into Delta (idempotent-friendly)
INSERT INTO sales_delta
SELECT 5 AS id, "2025-01-04" AS sale_date, "US" AS country, 500.0 AS amount;


In [0]:
%sql

SELECT * FROM sales_delta;

In [0]:
%sql
ALTER TABLE sales_delta ALTER COLUMN amount SET NOT NULL;
ALTER TABLE sales_delta ADD CONSTRAINT amount CHECK (amount > 0);

In [0]:
%sql
-- This insert will FAIL and nothing will be written
INSERT INTO sales_delta (id, sale_date, country, amount)
VALUES
  (100, '2025-01-10', 'UK', NULL),   -- violates NOT NULL
  (101, '2025-01-10', 'US', -50.0);  -- violates CHECK (amount > 0)


In [0]:
%sql
DESCRIBE HISTORY sales_delta;

In [0]:
%sql
ALTER TABLE sales_delta
  SET TBLPROPERTIES (delta.enableChangeDataFeed = true);


In [0]:
%sql
-- 1) New row
INSERT INTO sales_delta (id, sale_date, country, amount)
VALUES (201, '2025-01-11', 'UK', 100.0);

-- 2) Update an existing row
UPDATE sales_delta
SET amount = amount + 50
WHERE id = 1;

-- 3) Delete a row
DELETE FROM sales_delta
WHERE id = 2;


In [0]:
%sql
DESCRIBE HISTORY sales_delta;

In [0]:
%sql

SELECT * FROM sales_delta;

In [0]:
%sql

DESCRIBE HISTORY sales_delta;


In [0]:
%sql
SELECT *
FROM table_changes('sales_delta', 6, 9);


In [0]:
%sql
CREATE OR REPLACE TABLE sales_silver
USING DELTA
AS
SELECT
  id,
  sale_date,
  country,
  amount
FROM sales_delta;


In [0]:
%sql
MERGE INTO sales_silver AS tgt
USING (
  SELECT *
  FROM table_changes('sales_delta', 6, 9)
  WHERE _change_type != 'update_preimage'
) AS src
ON tgt.id = src.id

WHEN MATCHED AND src._change_type = 'delete' THEN
  DELETE

WHEN MATCHED THEN
  UPDATE SET *

WHEN NOT MATCHED AND src._change_type = 'insert' THEN
  INSERT *;


In [0]:
%sql

Select * from sales_silver;